In [1]:
import pandas as pd
import numpy as np
import json
import re
from textblob import TextBlob
import matplotlib.pyplot as plt 
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.text import Text
import string, re
import math 
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.gensim
from gensim import corpora
import gensim

C:\Users\DongHyeok\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\Users\DongHyeok\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
pd.set_option('max_colwidth', 1000)
pd.options.display.max_rows = 1000
data = pd.read_csv("en_twitter.csv")
data = data[['text']]

In [3]:
# Data preprocessing
# 1. Extract data
# 2. Remove unnecessary things(link, emoticon) & Convert every words to lower case
# 3. Tokenize and Lemmatize text & remove stop words % stemming
# 4. Remove too short text(words<3) 
# 5. Create the TF-IDF matrix


# 1. Extract data
KLM = data[data['text'].str.contains('@klm',case=False)][['text']]
#singapore = data[data['text'].str.contains('@SingaporeAir',case=False)][['text']]

# 2. Remove unnecessary things(stopwords, link, emoticon) & stemming & Convert every words to lower case
def cleantxt(text):
    text = text.lower()
    text = re.sub('airline', '', text)
    text = re.sub('airport', '', text)
    text = re.sub('flight', '', text)
    text = re.sub('flying', '', text)
    text = re.sub('amsterdam', '', text)
    text = re.sub('please', '', text)
    text = re.sub('airways', '', text)
    text = re.sub('people', '', text)
    text = re.sub('next', '', text)
    text = re.sub('europe', '', text)
    text = re.sub('would', '', text)
    text = re.sub('dutch', '', text)
    text = re.sub('year', '', text)
    
    text = re.sub('\[.*?\]', '', text) 
    text = re.sub(r'@[A-Za-z0-9]+', '', text) 
    text = re.sub(r'https?:\/\/\S+', '', text) 
    text = re.sub(r'RT[\s]+', '', text) 
    text = re.sub(r'#[A-Za-z0-9]+', '', text)  
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\n', '', text)
    return text

KLM['text'] = KLM['text'].apply(cleantxt)
#singapore['text'] = singapore['text'].apply(cleantxt)

# Remove emoji
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

KLM['text'] = KLM['text'].apply(deEmojify)
#singapore['text'] = singapore['text'].apply(deEmojify)

<>:30: DeprecationWarning: invalid escape sequence \[
<>:36: DeprecationWarning: invalid escape sequence \w
<>:30: DeprecationWarning: invalid escape sequence \[
<>:36: DeprecationWarning: invalid escape sequence \w
<>:30: DeprecationWarning: invalid escape sequence \[
<>:36: DeprecationWarning: invalid escape sequence \w
<ipython-input-3-8ac51dd2b847>:30: DeprecationWarning: invalid escape sequence \[
  text = re.sub('\[.*?\]', '', text)
<ipython-input-3-8ac51dd2b847>:36: DeprecationWarning: invalid escape sequence \w
  text = re.sub('\w*\d\w*', '', text)


In [4]:
# Get most frequent words and remove if meaningless
from collections import Counter
cnt = Counter()
for text in KLM["text"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(50)

[('to', 57033),
 ('the', 47883),
 ('a', 32151),
 ('and', 31357),
 ('i', 29103),
 ('you', 24303),
 ('is', 20642),
 ('of', 19613),
 ('for', 19189),
 ('in', 18599),
 ('my', 16487),
 ('on', 16247),
 ('with', 12959),
 ('it', 12775),
 ('that', 12458),
 ('have', 11725),
 ('your', 11653),
 ('this', 11646),
 ('not', 11516),
 ('are', 9827),
 ('from', 9370),
 ('be', 9258),
 ('they', 8172),
 ('but', 7954),
 ('me', 7881),
 ('was', 7603),
 ('s', 7449),
 ('at', 7380),
 ('klm', 7219),
 ('no', 6900),
 ('we', 6783),
 ('can', 6652),
 ('if', 6633),
 ('as', 6416),
 ('so', 5926),
 ('or', 5329),
 ('will', 5292),
 ('get', 5268),
 ('an', 5191),
 ('do', 5130),
 ('their', 5062),
 ('its', 5050),
 ('all', 5002),
 ('by', 4951),
 ('just', 4649),
 ('what', 4141),
 ('been', 4045),
 ('uk', 4038),
 ('about', 3956),
 ('dont', 3922)]

In [5]:
# 3. Tokenize and Lemmatize text 

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

KLM['text'] = KLM.text.apply(lemmatize_text)
#singapore['text'] = singapore.text.apply(lemmatize_text)

In [6]:
# remove stop words 

stop = stopwords.words('english')
KLM['text'] = KLM['text'].apply(lambda x: [word for word in x if word not in (stop)])
#singapore['text'] = singapore['text'].apply(lambda x: [word for word in x if word not in (stop)])


In [7]:
# 4. Remove too short text(words<3) 
tokenized_doc_k = KLM['text'].apply(lambda x: [word for word in x if len(word) > 3])
#tokenized_doc_s = singapore['text'].apply(lambda x: [word for word in x if len(word) > 3])

In [8]:
# Reset index for detokenization
tokenized_doc_k.reset_index(drop=True, inplace=True)
#tokenized_doc_s.reset_index(drop=True, inplace=True)

In [9]:
# 5. Create the TF-IDF matrix

# Detokenization
detokenized_doc_k = []
for i in range(len(KLM)):
    t = ' '.join(tokenized_doc_k[i])
    detokenized_doc_k.append(t)

KLM['text'] = detokenized_doc_k

vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000) # Preserve the top 1000 words
X_1 = vectorizer.fit_transform(KLM['text'])

X_1.shape 



(44514, 1000)

In [10]:
# Train Latent Dirichlet Allocation(LDA)

In [11]:
# Topic : 5 / Show the top important 5 words to determine the topic
lda_model=LatentDirichletAllocation(n_components=5,learning_method='online',random_state=777,max_iter=1)

lda_top=lda_model.fit_transform(X_1)
terms = vectorizer.get_feature_names() 
def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('baby', 515.68), ('breastfeeding', 470.33), ('offended', 408.36), ('cover', 405.38), ('mother', 402.51)]
Topic 2: [('voucher', 786.74), ('date', 369.27), ('refund', 360.39), ('country', 261.61), ('work', 250.58)]
Topic 3: [('hour', 433.73), ('email', 403.25), ('rebook', 352.09), ('customer', 331.99), ('response', 322.69)]
Topic 4: [('refund', 832.98), ('cancelled', 640.01), ('change', 613.27), ('help', 607.38), ('phone', 555.38)]
Topic 5: [('return', 314.49), ('hello', 262.58), ('possible', 228.16), ('press', 224.61), ('charge', 212.16)]


In [12]:
# Train Latent Dirichlet Allocation(LDA)

dictionary = corpora.Dictionary(tokenized_doc_k)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc_k]

In [17]:
# Topic : 6
NUM_TOPICS = 4 # Topics = 6
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.007*"make" + 0.006*"report" + 0.006*"avoid" + 0.006*"better"')
(1, '0.018*"baby" + 0.015*"breastfeeding" + 0.013*"passenger" + 0.013*"dont"')
(2, '0.013*"time" + 0.009*"seat" + 0.009*"back" + 0.009*"luggage"')
(3, '0.018*"refund" + 0.016*"help" + 0.014*"change" + 0.013*"customer"')


In [18]:
# t-SNE visualization
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
import matplotlib.colors as mcolors

# Get topic weights

topic_weights = []
for row_list in ldamodel[corpus]:
    tmp = np.zeros(4)
    for i, w in row_list:
        tmp[i] = w
    topic_weights.append(tmp)
arr = pd.DataFrame(topic_weights).fillna(0).values

# Array of topic weights    
arr = pd.DataFrame(topic_weights).fillna(0).values

# Keep the well separated points (optional)
arr = arr[np.amax(arr, axis=1) > 0.35]

# Dominant topic number in each doc
topic_num = np.argmax(arr, axis=1)

# tSNE Dimension Reduction
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
tsne_lda = tsne_model.fit_transform(arr)

# Plot the Topic Clusters using Bokeh
output_notebook()
n_topics = 4
mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), 
              plot_width=900, plot_height=700)
plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])
show(plot)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 43957 samples in 0.052s...
[t-SNE] Computed neighbors for 43957 samples in 1.115s...
[t-SNE] Computed conditional probabilities for sample 1000 / 43957
[t-SNE] Computed conditional probabilities for sample 2000 / 43957
[t-SNE] Computed conditional probabilities for sample 3000 / 43957
[t-SNE] Computed conditional probabilities for sample 4000 / 43957
[t-SNE] Computed conditional probabilities for sample 5000 / 43957
[t-SNE] Computed conditional probabilities for sample 6000 / 43957
[t-SNE] Computed conditional probabilities for sample 7000 / 43957
[t-SNE] Computed conditional probabilities for sample 8000 / 43957
[t-SNE] Computed conditional probabilities for sample 9000 / 43957
[t-SNE] Computed conditional probabilities for sample 10000 / 43957
[t-SNE] Computed conditional probabilities for sample 11000 / 43957
[t-SNE] Computed conditional probabilities for sample 12000 / 43957
[t-SNE] Computed conditional probabilities for sam

Loading BokehJS ...

In [19]:
# Visualization(KLM)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\DongHyeok\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
